# Kaggle Instacart Dataset
# The project contains 6 datasets for which I'll provide the description.

In [1]:
# import all the necessary libraries in the notebook to process the datasets

# 1st import pandas to convert .csv files in pandas dataframe
import pandas as pd

# 2nd I'm importing DASK-a flexible parallel computing library for analytic computing
import dask.dataframe as dd

# 3rd Python 2D plotting library 
import matplotlib.pyplot as plt

# 4rd Dask supports a real-time task framework that extends Python’s concurrent.futures interface. 
# This interface is good for arbitrary task scheduling, like dask.delayed, but is immediate rather than lazy, which provides some more flexibility in situations where the computations may evolve over time.
from dask.distributed import Client


# 5th Get matplotlib graphics to show up inline
%matplotlib inline

# File 1 aisles.csv
# Description: aisle_id (primary key),aisle(aisles tagged)

In [2]:
%%time 
aisles=pd.read_csv(r'C:\Users\abhij\Google Drive\Springboard\Capstone Datasets\Instacart Dataset\aisles.csv',index_col=None,dtype={"aisle_id":"int64","aisle":"object"})

Wall time: 27 ms


In [3]:
aisles.head()

,aisle_id,aisle
0,1,prepared soups salads
1,2,specialty cheeses
2,3,energy granola bars
3,4,instant foods
4,5,marinades meat preparation


In [4]:
#Set index operation to replace line number with aisle_id for efficient access

In [5]:
aisles1=aisles.set_index(['aisle_id'])

In [6]:
aisles1.head()

,aisle
aisle_id,
1,prepared soups salads
2,specialty cheeses
3,energy granola bars
4,instant foods
5,marinades meat preparation


In [7]:
##To count the number of aisles listed in the dataset
aisles1.count()

aisle    134
dtype: int64

# File 2 departments.csv
# Description :department_id (primary key),department (department tagged)

In [8]:
depart=pd.read_csv(r'C:\Users\abhij\Google Drive\Springboard\Capstone Datasets\Instacart Dataset\departments.csv',index_col=None,dtype={"department_id":"int64","department":"object"})

In [9]:
depart.head()

,department_id,department
0,1,frozen
1,2,other
2,3,bakery
3,4,produce
4,5,alcohol


In [10]:
#Set index operation to replace line number with department_id for faster access

In [11]:
depart1=depart.set_index(['department_id'])

In [12]:
depart1.head()

,department
department_id,
1,frozen
2,other
3,bakery
4,produce
5,alcohol


In [13]:
##Count the number of departments in the datasets
depart1.count()

department    21
dtype: int64

# File 3 order_products__prior
# Description: order_id(foreign key), product_id(foreign key), add_to_cart_order, reordered


In [14]:
#I'm using dask dataframe over here

order_prod_prior=dd.read_csv(r'C:\Users\abhij\Google Drive\Springboard\Capstone Datasets\Instacart Dataset\order_products__prior.csv')

In [15]:
#used Dask dataframe here for quick access
order_prod_prior.head(20)

,order_id,product_id,add_to_cart_order,reordered
0,2,33120,1,1
1,2,28985,2,1
2,2,9327,3,0
3,2,45918,4,1
4,2,30035,5,0
5,2,17794,6,1
6,2,40141,7,1
7,2,1819,8,1
8,2,43668,9,0
9,3,33754,1,1


In [19]:
orderprod1=order_prod_prior.set_index('order_id')
orderprod1.head()

,product_id,add_to_cart_order,reordered
order_id,,,
2,33120,1,1
2,28985,2,1
2,9327,3,0
2,45918,4,1
2,30035,5,0


In [17]:
%%time
client=Client()

Wall time: 7.87 s


In [18]:
client

<Client: scheduler='tcp://127.0.0.1:49929' processes=8 cores=8>

In [31]:
%%time
orderprod1.count().compute()

Wall time: 42.9 s


product_id           32434489
add_to_cart_order    32434489
reordered            32434489
dtype: int64

# File 4 orders.csv


In [ ]:
# Description: order_id:primary key
#            : user_id:unique users
#            : eval_set:prior vs train
#            : order_number:sequence of order
#            : order_dow: day of week on which the order was placed 
#            : order_hour_of_day: Hour of the day
#            : days_since_prior_order:Days since the previous order was placed (latency)

In [20]:
%%time
orders=dd.read_csv(r'C:\Users\abhij\Google Drive\Springboard\Capstone Datasets\Instacart Dataset\orders.csv',dtype={"order_id":"int64","user_id":"int64","eval_set":"object","order_number":"int64","order_dow":"int64","order_hour_of_day":"int64","days_since_prior_order":"float64"})

Wall time: 218 ms


In [23]:
orders.head(4)

,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order
0,2539329,1,prior,1,2,8,NaN
1,2398795,1,prior,2,3,7,15.0
2,473747,1,prior,3,3,12,21.0
3,2254736,1,prior,4,4,7,29.0


In [24]:
orders2=orders.set_index('order_id')
orders2.head(5)


,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order
order_id,,,,,,
1,112108,train,4,4,10,9.0
2,202279,prior,3,5,9,8.0
3,205970,prior,16,5,17,12.0
4,178520,prior,36,1,9,7.0
5,156122,prior,42,6,16,9.0


In [25]:
%%time
client = Client()

Wall time: 28.8 s


In [26]:
client

<Client: scheduler='tcp://127.0.0.1:50027' processes=8 cores=8>

In [27]:
%%time
mean1=client.compute(orders2.order_hour_of_day.mean())

Wall time: 7 ms


In [28]:
mean1.result()

13.45201534134074

In [29]:
%%time
orders2.count().compute()

Wall time: 5.83 s


user_id                   3421083
eval_set                  3421083
order_number              3421083
order_dow                 3421083
order_hour_of_day         3421083
days_since_prior_order    3214874
dtype: int64

In [30]:
%%time
find_null=client.compute(orders2.days_since_prior_order.isnull())

Wall time: 6 ms


# File 5 products.csv


In [32]:
#Description: product_id:primary key
#           : product_name:product description
#           : aisle_id:foreign key
#           : department_id: foreign key

In [33]:
%%time
products=pd.read_csv(r'C:\Users\abhij\Google Drive\Springboard\Capstone Datasets\Instacart Dataset\products.csv',dtype={"product_id":"int64","product_name":"object","aisle_id":"int64","department_id":"int64"})

Wall time: 290 ms


In [35]:
products.head()

,product_id,product_name,aisle_id,department_id
0,1,Chocolate Sandwich Cookies,61,19
1,2,All-Seasons Salt,104,13
2,3,Robust Golden Unsweetened Oolong Tea,94,7
3,4,Smart Ones Classic Favorites Mini Rigatoni Wit...,38,1
4,5,Green Chile Anytime Sauce,5,13


In [36]:
#Set index operation to remove line number and replace it with product_id aisle_id & department_id

In [37]:
products1=products.set_index(['product_id','aisle_id','department_id'])

In [38]:
products1.head()

,,,product_name
product_id,aisle_id,department_id,
1,61,19,Chocolate Sandwich Cookies
2,104,13,All-Seasons Salt
3,94,7,Robust Golden Unsweetened Oolong Tea
4,38,1,Smart Ones Classic Favorites Mini Rigatoni Wit...
5,5,13,Green Chile Anytime Sauce


# Join aisles1 and products1 on aisle_id

In [39]:
%%time
result_df=pd.merge(aisles1,products1,right_index=True,left_index=True)

Wall time: 6 ms


In [40]:
%%time
result_df.head()

Wall time: 1e+03 µs


,,,aisle,product_name
product_id,aisle_id,department_id,,
1,61,19,cookies cakes,Chocolate Sandwich Cookies
2,104,13,spices seasonings,All-Seasons Salt
3,94,7,tea,Robust Golden Unsweetened Oolong Tea
4,38,1,frozen meals,Smart Ones Classic Favorites Mini Rigatoni Wit...
5,5,13,marinades meat preparation,Green Chile Anytime Sauce


# Join result_df (key department_id) and depart1 (key deparment_id)

In [43]:
%%time
result_df1=pd.merge(result_df,depart1,right_index=True,left_index=True)

Wall time: 4 ms


# So  finally we have a joined pandas dataframe with files   aisles.csv, departments.csv, products.csv

In [44]:
result_df1.head(6)

,,,aisle,product_name,department
product_id,aisle_id,department_id,,,
1,61,19,cookies cakes,Chocolate Sandwich Cookies,snacks
2,104,13,spices seasonings,All-Seasons Salt,pantry
3,94,7,tea,Robust Golden Unsweetened Oolong Tea,beverages
4,38,1,frozen meals,Smart Ones Classic Favorites Mini Rigatoni Wit...,frozen
5,5,13,marinades meat preparation,Green Chile Anytime Sauce,pantry
6,11,11,cold flu allergy,Dry Nose Oil,personal care


In [57]:
result_new=orderprod1.merge(orders2,how='left',on=order_id)

NameError: name 'order_id' is not defined